# Section 1 : VectorDB Operations with Faiss

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQAWithSourcesChain
import pandas as pd
import os
from dotenv import load_dotenv # openai api key 입력을 위해서 필요

In [2]:
OPENAI_API_KEY_PATH = "/Users/eugene/Dropbox/0_Dev/07_LLM/project/env_folder/tonchat_key/.env"
# 셋째, .env 파일을 현재 실행환경에 등록. 그 결과, 개별 API 함수의 인자에 KEY값을 일일이 넣지 않아도 됨
load_dotenv(OPENAI_API_KEY_PATH)
# 넷째, 현재 실행환경에서 "OPENAI_API_KEY"라고 명명된 값을 불러오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# DirectoryLoader 객체 생성
loader = DirectoryLoader("./data/", loader_cls=PyPDFLoader)

# load_and_split() 메소드를 호출하여 문서를 로드하고 page 단위로 분할저장
pages = loader.load_and_split()

# CharacterTextSplitter 객체 생성
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# page 단위로 저장된 문서를 일정갯수의 character 갯수로 분할 후, document 단위로 분할저장
docs = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()


/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
# 현재 파이썬 라이브러리가 설치된 장소를 알아보자
import sys
sys.path

['/Users/eugene/Dropbox/0_Dev/07_LLM/project/tonchat',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python310.zip',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/lib-dynload',
 '',
 '/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages']

In [6]:
# 위의 경로에 패키지들이 제대로 설칭되었는지 확인
!pip list -v

Package                                  Version    Location                                                          Installer
---------------------------------------- ---------- ----------------------------------------------------------------- ---------
aiofiles                                 23.2.1     /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
aiohttp                                  3.9.3      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
aiosignal                                1.3.1      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
altair                                   5.2.0      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
annotated-types                          0.6.0      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
anyio                                    4.2.0      /Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages pip
appnope     

In [7]:
# docs에 저장된 문서들을 embeddings변수에 저장해둔 구체적인 embedding model을 이용하여 vector DB를 생성
db = FAISS.from_documents(docs, embeddings)

## 01. similarity search

In [8]:
# similarity_search(query) 메소드를 이용 : SQL언어가 아닌 자연어인 query와 가장 유사한 문서를 찾아냄
query = 'fee'
docs_result = db.similarity_search(query)
docs_result

[Document(page_content="24. 1. 4. 오전  10:49 L2 fee - (current) User Guide\nhttps://tokamaknetwork.gitbook.io/home/02-service-guide/titan/user-guide/l2-fee 1/1L2 fee\nThis document outlines the calculation process for transaction fees on Titan, emphasizing their lower\ncost compared to Ethereum.\nTitan's transaction fees are determined by adding two components together:\nL2 execution fee\nL1 security fee\nL2 execution fee\ntx.gasPrice * l2GasUsed\nThe L2 execution fee refers to the fee incurred when a user's transaction is processed within the\nTitan layer2 network. It follows a calculation method shown above, identical to the standard\nEthereum transaction fee computation. However, the distinction lies in the fact that the gas price on\nTitan is significantly lower compared to Ethereum.\nL1 security fee\nl1_base_fee * (tx_data_gas + overhaed) * scalar\nThe L1 security fee is an essential fee required in the context of Optimistic Rollup. With Optimistic\nRollup, all L2 transactions are 

In [9]:
# db검색을 위한 vector DBQA 객체를 생성
retriever = db.as_retriever()
# OpenAI(model="gpt-3.5-turbo-instruct") 처럼 ()안에 사용할 모델을 명시하지 않으면 text-davinci-003을 사용하는데, 이는 legacy model이라 헌재 지원안됨
# 따라서 OpenAI(model="gpt-3.5-turbo-instruct") 처럼 ()안에 사용할 모델을 명시해야 함
# 지시사항을 수행하는 용도인 InstructGPT을 선택해야 하며, 대화용도의 ChatGPT 모델을 선택하면 안됨
model = RetrievalQAWithSourcesChain.from_chain_type(llm=OpenAI(model="gpt-3.5-turbo-instruct"), chain_type="stuff", retriever=retriever)

/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [10]:
question = "metamask에 titan 네트워크를 추가하는 방법은 알려주세요 "

# model() 메소드는 langchain 라이브러리의 RetrievalQAWithSourcesChain 클래스의 인스턴스 메서드
# 질의응답, context 등은 dictionary 자료형임
response = model({"question": question}, return_only_outputs=True)
print("Answer: ", response['answer'])
print("Source", response['sources'])

/Users/eugene/opt/anaconda3/envs/LLM/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer:   To add the Titan network in Metamask, go to the Chrome web store, search for MetaMask, and click on 'Add to Chrome'. Then, click on the 'Add to Chrome' button and the network will be successfully added. 

Source data/Add Titan Network in Metamask_KR.pdf


## 02. Saving & loading

In [12]:
# 폴더명 지정
save_directory = "test_vectordb"
# save_local 메소드는 "test_vectordb" 폴더가 없으면 erro 출력대신 해당폴더를 생성한다는 장점이 있다
# 해당폴더안에 기존의 index.faiss, index.pkl 파일이 있는 경우에는 덮어쓴다
db.save_local(save_directory)

In [13]:
# saving, loading 둘다 폴더명을 인자로 받는다는 점을 주의해야 함 (파일명 x)
new_db = FAISS.load_local(save_directory, embeddings)


In [14]:
type(new_db)

langchain_community.vectorstores.faiss.FAISS

In [15]:
# db가 제대로 저장되었는지, 변경되었는지 파악하기 위해서 size를 측정해보는 것이 좋다
import sys
size_1 = sys.getsizeof(new_db)
size_2 = sys.getsizeof(db)
print("Size of new_db:", size_1, "bytes")
print("Size of db:", size_2, "bytes")


Size of new_db: 48 bytes
Size of db: 48 bytes


In [16]:
print(dir(new_db))

['_FAISS__add', '_FAISS__from', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_aembed_documents', '_aembed_query', '_asimilarity_search_with_relevance_scores', '_cosine_relevance_score_fn', '_embed_documents', '_embed_query', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_normalize_L2', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_embeddings', 'add_texts', 'adelete', 'afrom_documents', 'afrom_embeddings', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_search_by_vector', 'amax_marginal_

## 03. View documenents

FAISS.from_documents(docs, embeddings) 생성자를 통해 생성된 FAISS 객체의 자료구조를 확인 --> vectorstore update에 필요

In [17]:
# 일단 dataframe으로 변환을 통해,표 형태로 data를 보여줄 수 있도록 시각화  : display() 사용
def show_vstore(_store):
    vector_df = store_to_df(_store)
    display(vector_df)

# vectorstore는 dictionay 자료형
# FAISS.load_local()로 vectorstore 객체의 docstore속성의 하위 속성인 _dict 속성을 이용하여 dictionary형으로 자료를 추출해야 함
# dictionary의 key값은 chunk_id 이므로 .keys() 메소드를 이용하여 추출 (for문을 이용하여 순차적으로 추출)
# value값은 metadata 속성이며, 그 하위에 document, page, content 등으로 존재함
# 먼저 dictionary의 metadata에서 source, page, page_content 등 3가지 value를 추출한 후
# name, page number, content 등의 이름으로 저장 --> 이를 data_rows에 추가
def store_to_df(_store):
    v_dict = _store.docstore._dict
    data_rows = []
    for k in v_dict.keys():
        doc_name = v_dict[k].metadata['source'].split('/')[-1]
        page_number = v_dict[k].metadata['page']+1
        content = v_dict[k].page_content
        # key값과 함께 3개의 value값을 dictionary로 만들어서 data_rows에 추가
        data_rows.append({'chunk_id': k, 'document': doc_name, 'page': page_number, 'content': content})
    vector_df = pd.DataFrame(data_rows)
    return vector_df

# '''
# metadata는 faiss의 데이터 구조인 vectorstore의 일부로서, 그 안에는 사전(dictionary) 형태의 데이터 구조를 가짐
# 사전의 key, value구조를 통해 문서의 출처(source), 페이지 번호(page)와 같은 문서에 대한 중요한 정보를 포함
# 이 정보를 통해 각 데이터 항목이 어디에서 왔는지, 어떤 특성을 가지고 있는지 등을 파악할 수 있습니다.
# '''

In [18]:
show_vstore(new_db)

,chunk_id,document,page,content
0,7a2f1579-462b-4a55-b79e-97a926745cbb,L2 fee.pdf,1,24. 1. 4. 오전 10:49 L2 fee - (current) User Gu...
1,5dacdfe6-c015-4eb6-9634-17bcd5c87a75,What is different_KR.pdf,1,24. 1. 4. 오후 1:01 What is different - (curren...
2,977fb681-3f4f-4963-a782-a7d78829f82e,What is different_KR.pdf,2,24. 1. 4. 오후 1:01 What is different - (curren...
3,f684bc53-6833-41ac-9572-bdd71b21d003,Developer Guide - (current) User Guide.pdf,1,24. 1. 4. 오전 10:50 Developer Guide - (current...
4,e23fb206-e001-4119-b133-74da3fb799b8,Add Titan Network in Metamask_KR.pdf,1,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
5,1c83e76f-5c50-43b5-912d-c3e2c7d948e0,Add Titan Network in Metamask_KR.pdf,2,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
6,aa1c018a-01cc-46a8-820d-6ac182d637e2,Add Titan Network in Metamask_KR.pdf,3,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
7,d874c6f8-d76a-4e90-8c47-ebf9e762637b,Add Titan Network in Metamask_KR.pdf,4,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
8,9c42eca5-05b2-4e0a-92b9-6a62ec8905a3,Add Titan Network in Metamask_KR.pdf,5,24. 1. 4. 오후 12:58 Add Titan Network in Metam...
9,37c5d523-b2ad-43ea-b1a3-fc40f27bad80,Add Titan Network in Metamask_KR.pdf,6,24. 1. 4. 오후 12:58 Add Titan Network in Metam...


## 04. Delete documents from the database


In [30]:
# vector DB의 문서는 여러 개의 chunk로 쪼개져 있으므로 하나의 문서를 삭제하려면 ?
# 첫째, vector DB를 dataframe으로 변환
# 둘째, dataframe 메소드를 활용하여 삭제할 문서의 제목으로 검색하여 관련 chunk_id를 모두 찾아냄
# 셋째, vector DB 메소드를 활용하여 해당 chunk_id를 모두 삭제

def delete_document(_store,_document):
    # 우선 vectorstore 자료를 store_to_df() 함수를 이용하여 dataframe으로 변환
    vector_df = store_to_df(_store)
    # 삭제할 문서가 여러개의 chunk_id로 나뉘어져 있을 수 있으므로, 이를 모두 삭제해야 함
    # 삭제할 문서의 이름과 일치하는 문서의 모든 chunk_id만 추출해서 pandas의 tolist()메소드를 이용하여 list자료형으로 chunk_list에 저장
    chunk_list = vector_df.loc[vector_df['document']==_document]['chunk_id'].tolist()
    # FAISS 객체의 .delete() 메소드를 이용하여 chunk_list 해당부분 삭제
    _store.delete(chunk_list)

# model update (via retriever and chain)
def refresh_model(_new_store):
    retriever = _new_store.as_retriever()
    model = RetrievalQAWithSourcesChain.from_chain_type(llm=OpenAI(model="gpt-3.5-turbo-instruct"), chain_type="stuff", retriever=retriever)
    return model


In [31]:
delete_document(new_db, "Gas Estimation.pdf")
show_vstore(new_db)

,chunk_id,document,page,content
0,39d45e1a-ca40-4e69-a268-2e02607fa808,L2 fee.pdf,1,24. 1. 4. 오전 10:49 L2 fee - (current) User Gu...
1,703f3d0e-82ed-4f59-8caa-29c61f46e71c,Running a local development environment_KR.pdf,1,24. 1. 4. 오후 1:01 Running a local development...
2,6cc06eae-3cc3-4f71-b826-a32ef69c454d,Running a local development environment_KR.pdf,2,24. 1. 4. 오후 1:01 Running a local development...
3,e6aaf1a3-135a-43a0-9972-56a277b754e3,Running a local development environment_KR.pdf,3,24. 1. 4. 오후 1:01 Running a local development...
4,260c0b4d-1874-4444-b38d-340905244350,Contract Addresses.pdf,1,24. 1. 4. 오전 10:52 Contract Addresses - (curr...
...,...,...,...,...
75,cb2ad5bd-2656-411d-b7e9-b6af31459895,How to Create a Standard ERC20 Token in L2.pdf,3,24. 1. 4. 오전 10:50 How to Create a Standard E...
76,590c86ce-0caf-49aa-b275-fe58fb49533a,How to Create a Standard ERC20 Token in L2.pdf,4,24. 1. 4. 오전 10:50 How to Create a Standard E...
77,9ee0d69a-c54a-4bb8-a638-9826ee595c0e,How to Create a Standard ERC20 Token in L2.pdf,5,24. 1. 4. 오전 10:50 How to Create a Standard E...
78,3c9a43db-9d16-4e21-a58b-11f498a4ea76,How to Create a Standard ERC20 Token in L2.pdf,6,24. 1. 4. 오전 10:50 How to Create a Standard E...


In [32]:
# refresh_model

model = refresh_model(new_db)

In [ ]:
# check deletion
question = "explain fee briefly"
response = model({"question": question}, return_only_outputs=True)

print("Answer: ", response['answer'])
print("Source", response['sources'])

## 05. Add new document

In [ ]:
def add_to_vectorstore(directory, _store):
    loader = DirectoryLoader(directory, loader_cls=PyPDFLoader)
    pages = loader.load_and_split()
    docs = text_splitter.split_documents(pages)
    _store.add_documents(docs, embeddings)

## 06. 사용자 인증

배경 
1. openAI에서는 Oauth 관련 API를 제공하지 않으므로, openAI계정으로 로그인할 수 없다. 
2. openAI의 API를 이용한 서비스를 할 경우, 사용자는 자신의 API KEY를 가지고 다녀야 하는 것이 정책이다 (BYOK 정책)
3. 따라서 입력한 API KEY 값을 통해 
   1. 유효한 api key인지 판단
   2. 관리자의 api key인지 판단 (맞다면 관리자용 view를 제공)

In [5]:
import hashlib

# 1. 관리자의 api key의 hash값으로 변환하여 변수로 저장
admin_key = "120hx7gadvda12xh128998938495 " # "120hx7gadvda12xh128998938495"는 admin key를 가정한 임의의 문자열임
admin_key_hash = hashlib.sha256(admin_key.encode()).hexdigest()

print(admin_key_hash)



290038dd71afed14b4d42132fc498988aa4fc1142cf6972169aecd531af2c9fa


In [2]:
# 2. 사용자가 접속하여 text input 창에 api key를 입력
user_input_api_key = input("Enter your API key: ")

In [3]:
# 3. 해당 api key의 hash값이 관리자의 api key hash값과 일치하는지 검사
user_api_key_hash = hashlib.sha256(user_input_api_key.encode()).hexdigest()

In [4]:
# 4. hash 일치 여부 판단
is_admin = (user_api_key_hash == admin_key_hash)

# 결과 출력
print("Is user API key correct?", is_admin)

Is user API key correct? False
